# Function Calling

Read:

https://platform.openai.com/docs/guides/function-calling

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models


In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv
import os

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()
os.getenv("MY_NAME")

ModuleNotFoundError: No module named 'openai'

## The basic sequence of steps for function calling is as follows:

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

![Alt text](first.png "function_calling")

![Alt text](second.png "function_calling")

https://www.linkedin.com/pulse/azure-openai-function-calling-tarun-sharma/

In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


### Types

https://github.com/openai/openai-python/tree/main/src/openai/types/chat


In [6]:
import requests

data = requests.get("http://weather-forecast:8001") # use hostname/container name to access
print(data.text)

{"Welcome":"You can check weather of lahore & islamabad"}


In [7]:
print(data.status_code)
print(data.request)


200
<PreparedRequest [GET]>


In [16]:
import json
import requests
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from openai.types.chat.chat_completion import ChatCompletion, ChatCompletionMessage

_ = load_dotenv(find_dotenv()) # read local .env file
client: OpenAI = OpenAI()

def call_fastapi_get_current_weather(location:str, unit: str= "fahrenheit")->str:
    url = "http://weather-forecast:8001/get_current_weather"
    payloads = {"location": location, "unit":unit}
    response =requests.post(url,json=payloads)
    return json.dumps(response.json())

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location", "unit"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_book_flight",
                "description": "Book a flight for a specific destination and date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination": {
                            "type": "string",
                            "description": "The destination city, e.g. Paris, Tokyo",
                        },
                        # "date": {
                        #     "type": "string",
                        #     "format": "date",
                        #     "description": "The date of the flight, e.g. 2024-09-30",
                        # },
                        "passenger_count": {
                            "type": "integer",
                            "description": "The number of passengers for the flight",
                            "default": 1,
                        },
                    },
                    "required": ["destination", "passenger_count"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))
    print("---------------")

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))
    print("---------------")
    
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        # available_functions = {
        #     "get_current_weather": get_current_weather,
        #     "get_book_flight": get_book_flight
        # }  # only one function in this example, but you can have multiple
        # available_functions["get_current_weather"]()
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            function_response = call_fastapi_get_current_weather(
                location=function_args.get("location"),
                unit=function_args.get("unit")    
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
            # elif function_name == "get_current_weather":
            #     function_response = function_to_call(
            #     location=function_args.get("location"),
            #     unit=function_args.get("unit"),
            #     )
            #     messages.append(
            #     {
            #         "tool_call_id": tool_call.id,
            #         "role": "tool",
            #         "name": function_name,
            #         "content": function_response,
            #     }
            # )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        print("---------------")
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content
run_conversation("what the weather in lahore, muree and islamabad?")

'* First Response: '

{'content': None,
 'refusal': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_Xbn7OcI7GTIxpbA6C1ZzEkrq', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_pvmgXmPI4PgpuIdlQJ6XJXYO', function=Function(arguments='{"location": "Murree", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_G3EpL0YUUd48NU1rA3spNs00', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function')]}

---------------


'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_Xbn7OcI7GTIxpbA6C1ZzEkrq', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_pvmgXmPI4PgpuIdlQJ6XJXYO', function=Function(arguments='{"location": "Murree", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_G3EpL0YUUd48NU1rA3spNs00', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function')]

---------------


'* Second Request Messages: '

[{'role': 'user',
  'content': 'what the weather in lahore, muree and islamabad?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Xbn7OcI7GTIxpbA6C1ZzEkrq', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_pvmgXmPI4PgpuIdlQJ6XJXYO', function=Function(arguments='{"location": "Murree", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_G3EpL0YUUd48NU1rA3spNs00', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_Xbn7OcI7GTIxpbA6C1ZzEkrq',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '"{\\"location\\": \\"Lahore\\", \\"temperature\\": \\"72\\", \\"unit\\": \\"fahrenheit\\"}"'},
 {'tool_call_id': 'call_pvmgXmPI4PgpuIdlQ

---------------
* Second Response:  {'id': 'chatcmpl-A7i618nJaeIlixRRcGhoMfBPz0oTe', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Currently, the weather in Lahore is 72°F, in Murree it is 10°C, and in Islamabad it is 22°C.', refusal=None, role='assistant', function_call=None, tool_calls=None))], 'created': 1726401165, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': 'fp_54b3a4486f', 'usage': CompletionUsage(completion_tokens=29, prompt_tokens=177, total_tokens=206, completion_tokens_details={'reasoning_tokens': 0})}


'Currently, the weather in Lahore is 72°F, in Murree it is 10°C, and in Islamabad it is 22°C.'